In [ ]:
from src import crawler, indexing, invertedindex, loadweights, preprocess, annotate, setting, utility
import json
import os
from tqdm import tqdm
import pandas as pd 
import pickle
import numpy as np
from collections import defaultdict



def create_data(dataframe_path):
    df = pd.read_csv(dataframe_path, sep =',')
    df['annotation'] = df['annotation'].apply(lambda x: x.strip("[]'"))
    df['image_details'] = df['image_details'] + ' ' + df['annotation']
    df.to_csv(r'./data/indexed_data.csv', index=False, encoding='utf-8') 
    documents = [doc for doc in df['image_details']]
    processed_documents = [preprocess.PreprocessText(doc).clean_text() for doc in documents]
    image_data = defaultdict(dict)
    for idx, row in df.iterrows():
        image_id = row['image_id']
        image_data[image_id]['image_source'] = row['image_source']
        image_data[image_id]['image_details'] = row['image_details']
        
    with open(r'./data/image_data.pkl', 'wb') as f:
        pickle.dump(image_data, f)


def mapping(query, doc):    
    ''' 
    - Map each query id with all the document ids.
    
    :param query: dictionary with queryid and query as key-value pair.
    :param doc: dictionary with docid and text as key-value pairs.
    
    :return: dictionary with queryid as key and list of document ids as value.
    '''    
    mappings = {}
    for key, value in query.items():
        for k, v in doc.items():
            doc_keys = list(doc.keys())
            doc_keys = [int(i) for i in doc_keys]
            mappings[int(key)] = doc_keys 
    return mappings
# links_path = r'./config/links.json'
# with open(links_path) as f:
#     data = json.load(f)
# print(data['links'])
# indexed_data = indexing.index_collection(data['links'])

directory_path = r'./images'

file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, file))]
label = []
image_name = []
# for path in tqdm(file_paths):
#     final_data = annotate.showresults(path)
#     if not final_data[1]:
#         image_name.append(path[77:].split('_', 1)[1])
#         label.append(" ")
#     else:
#         label.append(final_data[1])
#         image_name.append(path[77:].split('_', 1)[1])
        
# df_labels = pd.DataFrame(data = {'image_name': image_name, 
#                                 'annotation': label})

# df_labels = df_labels.rename(columns={"image_name":"original_image_name"})

# df_merged = pd.merge(indexed_data, df_labels, on = "original_image_name")

# df_merged.to_csv(r'./data/indexed_data.csv', index=False, encoding='utf-8')   

# path = r'./data/indexed_data.csv'

# save_image_data = create_data(path)

    


In [ ]:
file_paths

In [8]:
import pandas as pd

df  = pd.read_csv(r'F:\OneDrive\nuig\modules\mechanics of search\assignment_2\assignment_new_trial\assignment_2\data\indexed_data.csv')
df.shape

(2256, 8)

In [ ]:
import pickle
with open(r'./data/image_data.pkl', 'rb') as f:
        vectors = pickle.load(f)
        
vectors

In [ ]:
import pandas as pd 

df =pd.read_csv(r'F:\OneDrive\nuig\modules\mechanics of search\assignment_2\assignment_new_trial\assignment_2\data\indexed_data.csv')
image_data = defaultdict(dict)
for idx, row in df.iterrows():
    image_id = row['image_id']
    image_data[image_id]['image_source'] = row['image_source']
    image_data[image_id]['image_details'] = row['image_details']
    image_data[image_id]['photographer'] = row['photographer']
    image_data[image_id]['image_name'] = row['image_name']

image_data

In [ ]:
from src import crawler, indexing, invertedindex, loadweights, preprocess, annotate, setting, utility
import json
import os
from tqdm import tqdm
import pandas as pd 
import pickle
import numpy as np
from collections import defaultdict

import pandas as pd 

data= pd.read_csv(r'./data/indexed_data.csv')

def generate_index(doc_collection):
    index = invertedindex.InvertedIndexDocs(doc_collection)
    index.create_index()
    return index

documents= {image_id: image_data[image_id]['image_details'] for image_id in image_data}
inv_index = generate_index(documents)

with open(r'./data/inverted_index.pkl', 'wb') as f:
    pickle.dump(inv_index, f)


In [ ]:
with open(r'./data/image_data.pkl', 'rb') as f:
    image_data = pickle.load(f)
    
indexed_docs= {image_id: image_data[image_id]['image_details'] for image_id in image_data}
indexed_docs

def generate_index(doc_collection):
    index = invertedindex.InvertedIndexDocs(doc_collection)
    index.create_index()
    return index

inv_index = generate_index(indexed_docs)

# save inv_index as pickle

with open(r'./data/inverted_index.pkl', 'wb') as f:
    pickle.dump(inv_index, f)

for k, v in inv_index.get_index.items():
    print(f'the term is {k}, posting list is {v.postings} and document frequency is {v.document_frequency}')

In [ ]:
with open(r'./data/inverted_index.pkl', 'rb') as f:
    inv = pickle.load(f)
    
for k, v in inv.get_index.items():
    print(f'the term is {k}, posting list is {v.postings} and document frequency is {v.document_frequency}')

# Test query output

In [3]:
# import necessary libraries

import xml.etree.ElementTree as ET
import csv
from tqdm import tqdm
 
# utilities
from collections import defaultdict
import re
import numpy as np
import pandas as pd
from copy import deepcopy
import os
from itertools import chain
import operator
# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize.treebank import (TreebankWordTokenizer, TreebankWordDetokenizer)
from string import punctuation
from nltk import PorterStemmer
from collections import Counter

import math
from functools import lru_cache
import numpy as np
import math
import operator
from tqdm import tqdm
import itertools
from collections import Counter
from dataclasses import dataclass
from math import log
import pickle
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from num2words import num2words
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

class PreprocessText:
    def __init__(self, text):
        self.text = text
        self.update_stop_words =  ['stock', 'images', 'free', 'clip', 'openphoto.be', 
                            'openphoto.meEgret', 'Strikes', 'free', 'photographs', 'for', 'download', 
                            'creative', 'commons', 'licensed', 'images']
        
    def _tokenize(self):
        ''' 
        - tokenise the text using word_tokenize function.
        :return list of tokens
        '''
        tokenizer = TreebankWordTokenizer()
        self.tokens = tokenizer.tokenize(self.text)  
        return self.tokens
    
    def _remove_stopwords(self):
        '''  
        - Remove stop words from the tokens which are the most commonly used words in english text.
        
        :return list of tokens without stopwords
        
        '''
        stop_words = set(stopwords.words('english'))
        stop_words.update(self.update_stop_words)
        return [word for word in self.tokens if word.lower() not in stop_words]
    
    def _convert_num_text(self):
        '''
        - convert numbers to words if the token is a number.
        
        :return list of tokens with numbers converted to words.
        
        '''
        text_num = [num2words(word) if word.isnumeric() and word.isascii() else word for word in self.tokens]
        return text_num
    
    def _normalise_text(self):
        ''' 
        - Normalise the text by converting all the words to lower case and removing punctuations.
        
        :return list of normalised tokens.
        
        '''
        normalised_text = [word.lower() for word in self.tokens if word.isalpha()]
        return normalised_text  
    

    def clean_text(self):
        ''' 
        - Clean the text by tokenising, normalising, removing stopwords and stemming.
        
        :return list of cleaned tokens.
        
        '''
        self._tokenize()
        self.tokens = self._convert_num_text()
        self.tokens = self._normalise_text()
        self.tokens = self._remove_stopwords()
        return self.tokens  
    


class RunModel:
    def __init__(self, index, mapping):
        self._document_collection = index.documents_collection
        self._index = index.get_index
        self._mapping = mapping
        
    def rank(self, qid, query):
        '''
        - Clean queries and get the relevant documents for each query word.
        
        :param qid: query id
        :param query: query text
        
        :return: dictionary with docid and score as key-value pairs.
        '''
        query_words = self._clean_query_text(query)
        docs = self._relevant_docs(qid, query_words)

        scores = {}
        for docid in docs:
            scores[docid] = self._score_docs(docid, query_words)

        ranking = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        ranks = {docid: score for docid, score in ranking}
        return ranks

    def _clean_query_text(self, query):
        clean_query_text = [word for word in PreprocessText(query).clean_text()]
        return clean_query_text

    def _relevant_docs(self, qid, query_words):
        ''' 
        - Helper function to get the relevant documents for each query word.
        
        :param qid: query id
        :param query_words: list of cleaned query words
        
        :return: list of relevant documents for each query word.
        '''
        relevant_docs_set = set() # set to avoid duplicates
        # get the relevant documents for each query word
        for word in query_words:  # iterate over the query words
            if word in self._index: # check if the word is in the index
                for docid in self._index[word].postings: # get the docids for each query word
                    if docid in self._mapping[qid]: # check if the docid is relevant to the query
                        relevant_docs_set.add(docid) # add the docid to the set
        return list(relevant_docs_set)

class VSM(RunModel):
    def __init__(self, index, mapping):
        super().__init__(index, mapping)
        self._vocab = index.vocab
        self._collection_length = index.document_length
        self._vocab_count = index.vocab_count

    def _score_docs(self, docid: int, query_words) -> float:
        vocab = list(set(self._document_collection[docid]))
        vocab_count = len(vocab)

        doc_vector = np.zeros(vocab_count)
        for i, word in enumerate(vocab):
            doc_vector[i] = self._index[word].get_posting(docid).tfidf

        query_vector = np.zeros(vocab_count)
        counter = Counter(query_words)
        max_freq = counter.most_common(1)[0][1]
        for word in query_words:
            if word not in self._index:
                continue
            tf = (0.5 + (0.5 * (counter[word] / max_freq))) # 
            idf = log(self._collection_length / self._index[word].document_frequency)
            tfidf = tf * idf
            if word in vocab:
                wordidx = vocab.index(word)
                query_vector[wordidx] = tfidf
            else:
                query_vector = np.append(query_vector, tfidf)
                doc_vector = np.append(doc_vector, 0)
                
        cosine_similarity = np.dot(doc_vector, query_vector) / (np.linalg.norm(doc_vector) * np.linalg.norm(query_vector))
        return cosine_similarity

In [55]:
import pickle
import yaml 

query = "Antonello !@#!@#!%!@#$!@#!@$!@$ qeqweMichele"
# query = {1:query}
# clean_query = {doc_id: PreprocessText(doc_text).clean_text() for doc_id, doc_text in query.items()}
# clean_query[1] = ' '.join(clean_query[1])

with open(r'F:\OneDrive\nuig\modules\mechanics of search\assignment_2\streamlit\assignment2\data\stopwords.yml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

stopwords = set(config['stop words'])  
pattern = r'\b[A-Za-z]+\b'
query_words = re.findall(pattern, query)

# lowercase 
query_words = [word.lower() for word in query_words]
# remove stopwords
query_words = [word for word in query_words if word not in stopwords]

query = {1:query_words}
clean_query = {key: ' '.join(value) for key, value in query.items()}
clean_query

{1: 'antonello qeqwemichele'}

In [12]:
import pickle

with open(r'F:\OneDrive\nuig\modules\mechanics of search\assignment_2\assignment_new_trial\assignment_2\data\image_data.pkl', 'rb') as f:
    image_data = pickle.load(f)

image_data 
    

defaultdict(dict,
            {0: {'image_source': 'https://cdn.openphoto.net/thumbs2/volumes/spankey/20050209/opl_StumblingOffTheWeeds.jpg',
              'image_details': 'Charlie Cushing egret, bird stock free images, free clip art, openphoto.be openphoto.meEgret Strikes Fish .  free photographs for download.  creative commons licensed images.   bird',
              'photographer': 'Charlie Cushing',
              'image_name': 'image_0.jpg'},
             1837: {'image_source': 'https://cdn.openphoto.net/thumbs2/volumes/spankey/20050209/opl_StumblingOffTheWeeds.jpg',
              'image_details': 'Charlie Cushing egret, bird stock free images, free clip art, openphoto.be openphoto.meEgret Strikes Fish .  free photographs for download.  creative commons licensed images.   bird',
              'photographer': 'Charlie Cushing',
              'image_name': 'image_1837.jpg'},
             1: {'image_source': 'https://cdn.openphoto.net/thumbs2/volumes/spankey/20050209/opl_SeesTheFish.j

In [1]:
import pickle

with open(r'F:\OneDrive\nuig\modules\mechanics of search\assignment_2\assignment_new_trial\assignment_2\data\image_data.pkl', 'rb') as f:
    image_data = pickle.load(f)
    
with open(r'F:\OneDrive\nuig\modules\mechanics of search\assignment_2\assignment_new_trial\assignment_2\data\inverted_index.pkl', 'rb') as f:
    inv = pickle.load(f)   


In [4]:
import pandas as pd 
from src import  invertedindex
import json
import os
from tqdm import tqdm
import pandas as pd 
import pickle
import numpy as np
from collections import defaultdict

with open(r'./data/image_data.pkl', 'rb') as f:
    image_data = pickle.load(f)

documents= {image_id: image_data[image_id]['image_details'] for image_id in image_data}


def generate_index(doc_collection):
    index = invertedindex.InvertedIndexDocs(doc_collection)
    indexed_docs = index.create_index()
    return indexed_docs

inv_index = generate_index(documents)

with open(r'./data/inverted_index_docs.pkl', 'wb') as f:
    pickle.dump(inv_index, f)

In [5]:
print(type(inv_index))

<class 'NoneType'>


In [6]:
inv_index

In [ ]:



def mapping(query, doc):    
    ''' 
    - Map each query id with all the document ids.
    
    :param query: dictionary with queryid and query as key-value pair.
    :param doc: dictionary with docid and text as key-value pairs.
    
    :return: dictionary with queryid as key and list of document ids as value.
    '''    
    mappings = {}
    for key, value in query.items():
        for k, v in doc.items():
            doc_keys = list(doc.keys())
            doc_keys = [int(i) for i in doc_keys]
            mappings[int(key)] = doc_keys 
    return mappings


indexed_docs= {image_id: image_data[image_id]['image_details'] for image_id in image_data}
indexed_docs

In [38]:
map_doc_queries = mapping(clean_query, indexed_docs)

In [39]:
model = VSM(inv, map_doc_queries)

# loop through each query and perform ranking
ranking = {qid: model.rank(qid, query) for qid, query in clean_query.items()}

filter_rank = {k: v for k, v in ranking[1].items() if v > 0.1}
filter_rank

{69: 0.566287700835568,
 70: 0.3394587664085562,
 110: 0.33798663212287045,
 60: 0.32558901722829947}

In [41]:
urls= {image_id: image_data[image_id]['image_source'] for image_id in image_data}
urls

{19: 'https://cdn.openphoto.net/thumbs2/volumes/mike/openphoto_dot_net/2006_1_2_11156_7_OPL.jpg',
 27: 'https://cdn.openphoto.net/thumbs2/volumes/johnkhan/20070519/openphotonet_2005-08-06%20at%2015-45-38.jpg',
 46: 'https://cdn.openphoto.net/thumbs2/volumes/mike/openphoto_dot_net/2005_10_22_5180_4_OPL.jpg',
 53: 'https://cdn.openphoto.net/thumbs2/volumes/johnkhan/20070519/openphotonet_2005-08-06%20at%2015-27-57.jpg',
 60: 'https://cdn.openphoto.net/thumbs2/volumes/lello/20070409/openphotonet_03_-_Diamantini.jpg',
 69: 'https://cdn.openphoto.net/thumbs2/volumes/lello/20070409/openphotonet_02_-_Diamantini.jpg',
 70: 'https://cdn.openphoto.net/thumbs2/volumes/lello/20070628/openphotonet_08_-_Torre_Argentina.jpg',
 106: 'https://cdn.openphoto.net/thumbs2/volumes/Delphin/20060925/openphotonet_133_3390_IMG.JPG',
 110: 'https://cdn.openphoto.net/thumbs2/volumes/lello/20070409/openphotonet_01_-_Diamantini.jpg',
 148: 'https://cdn.openphoto.net/thumbs2/volumes/mike/openphoto_dot_net/2005_10_22_

In [56]:
result = defaultdict(dict)
for key, value in filter_rank.items():
    result[key]['score'] = value
    result[key]['url'] = urls[key]
    result[key]['image_name'] = image_data[key]['image_name']
    result[key]['photographer'] = image_data[key]['photographer']
    result[key]['image_id'] = key
    
result

defaultdict(dict,
            {69: {'score': 0.566287700835568,
              'url': 'https://cdn.openphoto.net/thumbs2/volumes/lello/20070409/openphotonet_02_-_Diamantini.jpg',
              'image_name': 'image_69.jpg',
              'photographer': 'Antonello Michele Mastinu',
              'image_id': 69},
             70: {'score': 0.3394587664085562,
              'url': 'https://cdn.openphoto.net/thumbs2/volumes/lello/20070628/openphotonet_08_-_Torre_Argentina.jpg',
              'image_name': 'image_70.jpg',
              'photographer': 'Antonello Michele Mastinu',
              'image_id': 70},
             110: {'score': 0.33798663212287045,
              'url': 'https://cdn.openphoto.net/thumbs2/volumes/lello/20070409/openphotonet_01_-_Diamantini.jpg',
              'image_name': 'image_110.jpg',
              'photographer': 'Antonello Michele Mastinu',
              'image_id': 110},
             60: {'score': 0.32558901722829947,
              'url': 'https://cdn.openpho

In [43]:
for key, value in result.items():
    print(f'the image name is {value["image_name"]}, the url is {value["url"]}, the score is {value["score"]}, the photographer is {value["photographer"]}')
    

the image name is image_69.jpg, the url is https://cdn.openphoto.net/thumbs2/volumes/lello/20070409/openphotonet_02_-_Diamantini.jpg, the score is 0.566287700835568, the photographer is Antonello Michele Mastinu
the image name is image_70.jpg, the url is https://cdn.openphoto.net/thumbs2/volumes/lello/20070628/openphotonet_08_-_Torre_Argentina.jpg, the score is 0.3394587664085562, the photographer is Antonello Michele Mastinu
the image name is image_110.jpg, the url is https://cdn.openphoto.net/thumbs2/volumes/lello/20070409/openphotonet_01_-_Diamantini.jpg, the score is 0.33798663212287045, the photographer is Antonello Michele Mastinu
the image name is image_60.jpg, the url is https://cdn.openphoto.net/thumbs2/volumes/lello/20070409/openphotonet_03_-_Diamantini.jpg, the score is 0.32558901722829947, the photographer is Antonello Michele Mastinu


In [11]:
import numpy as np 


x = np.random.rand(4,3)
y = np.sum(x, axis =1, keepdims=True)

y.shape

(4, 1)